In [1]:
from phe import paillier
import numpy as np
from openpyxl import load_workbook
import time

In [2]:
start_time = time.time()

In [3]:
wb = load_workbook("marketing_case.xlsx")
ws1 = wb["A1"]
ws2 = wb["B1"]
n = 50000

# Bank of data

In [4]:
class Bank:
  def __init__(self, public_key):
    self.records = []
    for i in range(2,n):
        if i%5000 == 0:
            print(i)
        phone_number = ws1["A" + str(i)].value
        spending = np.random.uniform(0, 1000)
        timestamp = np.random.randint(0, 1000)
        self.records.append((phone_number, public_key.encrypt(spending), timestamp))

# Advertisement Hub

In [5]:
class AdvertisementHub:
     def __init__(self, public_key):
         self.records = []
         for i in range(2, n):
             if i%5000 == 0:
                 print(i)
             phone_number = ws2["A" + str(i)].value
             advertisement_views = np.random.randint(0, 10)
             timestamp = np.random.randint(0, 1000)
             self.records.append((phone_number, public_key.encrypt(advertisement_views), timestamp))

# PS3I

In [6]:
class PS3I:
     def __init__(self, bank, advertisement_hub, public_key):
         self.bank = bank
         self.advertisement_hub = advertisement_hub
         self.public_key = public_key
     def intersect(self):
         result = []
         for bank_record in self.bank.records:
             for ad_record in self.advertisement_hub.records:
                 if bank_record[0] == ad_record[0]:
                     result.append((bank_record[0], bank_record[1], ad_record[1], bank_record[2]))
         return result
     def calculate_risk(self, epsilon):
         intersection_size = len(self.intersect())
         bank_size = len(self.bank.records)
         advertisement_hub_size = len(self.advertisement_hub.records)
         privacy_budget = bank_size * advertisement_hub_size / intersection_size
         risk_level = epsilon / privacy_budget
         return risk_level

 # Example usage:

In [7]:
public_key, private_key = paillier.generate_paillier_keypair()
bank = Bank(public_key)
advertisement_hub = AdvertisementHub(public_key)
ps3i = PS3I(bank, advertisement_hub, public_key)

5000
10000
15000
20000
25000
30000
35000
40000
45000
5000
10000
15000
20000
25000
30000
35000
40000
45000


# Perform homomorphic addition and multiplication to calculate the sum of spending and advertisement views

In [8]:
spending_sum = public_key.encrypt(0)
ad_views_sum = public_key.encrypt(0)
for bank_record in bank.records:
     spending_sum += bank_record[1]
for ad_record in advertisement_hub.records:
     ad_views_sum += ad_record[1]

# Perform homomorphic multiplication to calculate the product of spending and advertisement views

In [9]:
spending_ad_views_product = public_key.encrypt(0)
for bank_record in bank.records:
     for ad_record in advertisement_hub.records:
         if bank_record[0] == ad_record[0] and bank_record[2] == ad_record[2]:
             spending_ad_views_product += bank_record[1] * ad_record[1]

# Decrypt the homomorphically computed results

In [10]:
spending_sum = private_key.decrypt(spending_sum)
ad_views_sum = private_key.decrypt(ad_views_sum)
spending_ad_views_product = private_key.decrypt(spending_ad_views_product)
intersection = ps3i.intersect()
risk_level = ps3i.calculate_risk(0.1)
end_time = time.time()
print("Intersection size: ", len(intersection))
print("Risk level: ", risk_level)
print("time:", end_time-start_time)

Intersection size:  128
Risk level:  5.120409624577311e-09
time: 22654.31810235977
